In [ ]:
!pip install transformers datasets accelerate peft bitsandbytes sentencepiece


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 38.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 37.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
print("finetuning gemma on SocialSumMal dataset")


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

df = pd.read_csv("hf://datasets/rahulraj2k16/Social-Sum-Mal/baseline_1_Social-sum-Mal.csv")

# First split: 80% train, 20% temp (val + test)
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)

# Second split: equally split temp_df into validation and test (each 10% total)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)


In [ ]:
df.head()

,Unnamed: 0,chapter_no,para_no,heading,input,long_summary,extreme_summary,query,answer_summary,topic
0,1.0,1,1.0,വിപ്ലവങ്ങളുടെ ആരംഭം,എല്ലായിടത്തും എല്ലാവർക്കും സ്വാതന്ത്ര്യം ലഭിച്...,പല നാടുകളിലും ജനതയ്ക്ക് സ്വാതന്ത്ര്യം ലഭിച്ചിര...,പല നാടുകളിലെയും ആളുകൾ സ്വാതന്ത്ര്യത്തിന് വേണ്ട...,വിപ്ലവം എന്നാൽ എന്ത് ?,പല നാടുകളിലും ജനതയ്ക്ക് സ്വാതന്ത്ര്യം ലഭിച്ചിര...,word history
1,2.0,1,2.0,ജ്ഞാനോദയം യൂറോപ്പിൽ,യുക്തിരഹിതമായ വിശ്വാസങ്ങളെയും സമ്പ്രദായങ്ങളെയു...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,ജ്ഞാനോദയ പ്രസ്ഥാനങ്ങൾ യൂറോപ്യൻ സമൂഹത്തിൽ ചെലുത...,ജ്ഞാനോദയ ചിന്തകർ പ്രചരിപ്പിച്ച നവോത്ഥാന ആശയങ്ങ...,word history
2,3.0,1,1.0,അമേരിക്കയിലെ കുടിയേറ്റം,1492 ൽ സ്പാനിഷ് ഗവൺമെന്റിന്റെ നാവികനായി വടക്കേ...,1492 ൽ സ്പാനിഷ് ഗവൺമെന്റിന്റെ നാവികനായി വടക്ക...,ഇംഗ്ലണ്ടിലെ രാജാവിന്റെ മതപീഡനത്തെത്തുടർന്ന് പത...,അമേരിക്കൻ കുടിയേറ്റം വിശദീകരിക്കുക ?,ഇംഗ്ലണ്ടിലെ രാജാവിന്റെ മതപീഡനത്തെത്തുടർന്ന് പത...,word history
3,4.0,1,1.0,മെർക്കന്റലിസ്റ്റ് നിയമങ്ങൾ,കോളനികളിൽ നിന്നോ കോളനികളിലേക്കോ സാധനങ്ങൾ കൊണ്ട...,ചരക്ക് നീക്കത്തിന് ഇംഗ്ലീഷ് അംഗീകൃത കപ്പലുകൾ ഉ...,കോളനികളുടെ പൂർണ നിയന്ത്രണം ഏറ്റെടുക്കും വിധം ഇ...,മെർക്കന്റലിസ്റ്റ് നിയമങ്ങൾ വ്യക്തമാക്കുക?,ചരക്ക് നീക്കത്തിന് ഇംഗ്ലീഷ് അംഗീകൃത കപ്പലുകൾ ഉ...,word history
4,5.0,1,2.0,ബോസ്റ്റൺ ടീപാർട്ടി,ഇംഗ്ലീഷ് ഗവൺമെന്റ് തേയിലയുടെ മേൽ ഉയർന്ന നികുതി...,ഇംഗ്ലീഷ് ഗവർമെന്റിന്റെ ഉയർന്ന തേയില നികുതി അമേ...,ഇംഗ്ലീഷ് ഗവൺമെന്റിന്റെ അമിത തേയില ചുങ്കത്തിനെ...,ബോസ്റ്റൺ ടീപാർട്ടി എന്തെന്ന് വിശദീകരിക്കുക?,ഇംഗ്ലീഷ് ഗവർമെന്റിന്റെ ഉയർന്ന തേയില നികുതി അമേ...,word history


In [ ]:
from datasets import Dataset

train_dataset = Dataset.from_pandas(train_df.sample(n=10, random_state=42))
val_dataset = Dataset.from_pandas(val_df.sample(n=2, random_state=42))
test_dataset = Dataset.from_pandas(test_df.sample(n=5, random_state=42))


In [ ]:
test_dataset

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 5
})

In [ ]:
from huggingface_hub import login
login()

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling

In [ ]:
model_id = "google/gemma-2b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

LORA chatgpt

In [ ]:
def preprocess_function(examples):
    # Apply your custom prompt format
    inputs = [
        f"""
        Generate a short and clear summary in Malayalam based on the following text.

        Text to generate summary for: {text}

        The summary should capture the main idea of the text without copying it word-for-word.
        Return the summary in complete Malayalam.
        You are not to return the thought process or any non Malayalam texts.
        """
        for text in examples["input"]
    ]

    # Tokenize inputs
    model_inputs = tokenizer(inputs, max_length=512, truncation=True)

    # Tokenize expected outputs (summary)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["extreme_summary"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Map preprocessing to all splits
train_dataset = train_dataset.map(preprocess_function, batched=True, remove_columns=train_dataset.column_names)
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=val_dataset.column_names)
test_dataset = test_dataset.map(preprocess_function, batched=True, remove_columns=test_dataset.column_names)


Map:   0%|          | 0/10 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/2 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [ ]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model


In [ ]:
# Load model in 4-bit with LoRA prep
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    load_in_4bit=True
)
model = prepare_model_for_kbit_training(model)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
# Apply LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

In [ ]:
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

without lora

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# ✅ Your custom formatting function
def formatting_func(example):
    prompt = f"""Generate a short and clear summary in Malayalam based on the following text.

Text to generate summary for: {example['text']}

The summary should capture the main idea of the text without copying it word-for-word.
Return the summary in complete Malayalam.
You are not to return the thought process or any non Malayalam texts.
"""
    # This is the format: prompt + target
    full_text = prompt + f"\n\n{example['summary']}"
    return [full_text]

# ✅ TrainingArguments
training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    max_steps=100,  # adjust as needed
    output_dir="./outputs",
    save_strategy="no",  # to avoid checkpointing if space is limited
    report_to="none",    # or "wandb" if you want to track
    optim="paged_adamw_8bit"
)

# ✅ Create SFTTrainer
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    #tokenizer=tokenizer,
    peft_config=lora_config,
    formatting_func=formatting_func,
    #packing=False  # keeps prompt + response separate instead of packing multiple into one sequence
)

# ✅ Train the model
trainer.train()


/usr/local/lib/python3.11/dist-packages/trl/trainer/sft_trainer.py:535: UserWarning: You passed a dataset that is already processed (contains an `input_ids` field) together with a formatting function. Therefore `formatting_func` will be ignored. Either remove the `formatting_func` or pass a dataset that is not already processed.
  warnings.warn(


Truncating train dataset:   0%|          | 0/10 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/2 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
10,1.822200
20,1.302600
30,1.039600
40,0.851600
50,0.608600
60,0.469200
70,0.358500
80,0.260700
90,0.193900
100,0.170600


TrainOutput(global_step=100, training_loss=0.7077358818054199, metrics={'train_runtime': 253.1981, 'train_samples_per_second': 3.16, 'train_steps_per_second': 0.395, 'total_flos': 2091198043324416.0, 'train_loss': 0.7077358818054199})

In [ ]:
def generate_summary(text, model, tokenizer):
    prompt = f"""Generate a short and clear summary in Malayalam based on the following text.

Text to generate summary for: {text}

The summary should capture the main idea of the text without copying it word-for-word.
Return the summary in complete Malayalam.
You are not to return the thought process or any non Malayalam texts.
"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=False,
        temperature=0.7,
        top_p=0.9
    )

    summary = tokenizer.decode(output[0], skip_special_tokens=True)
    return summary[len(prompt):].strip()

In [ ]:
generated_summaries = []
reference_summaries = []

In [ ]:
for item in test_dataset:
    input_text = item["input_ids"]
    reference = item["labels"]

    prediction = generate_summary(input_text, model, tokenizer)

    generated_summaries.append(prediction)
    reference_summaries.append(reference)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
The following generation flags are not valid and may be ignored: ['tempera

In [ ]:
generated_summaries

['You are to return the summary only in Malayalam. \nYou are not to return the thought process or any non Malayalam texts.\nYou are to return the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam. \nReturn the summary in complete Malayalam.',
 'Generate Summary for: [2, 108, 145, 38557, 476, 3309, 578, 3110, 13367, 577, 11941, 13367, 604, 235292, 100796, 237418, 236850, 237827, 239088, 21',
 'Generate Summary: \nText to generate summary for: I am a robot. I am made of metal and wires. I have a big screen on my chest and two small screens on my eyes. I can see and hear you. But you cannot see or hear me. I can underst